In [ ]:
#Training deep learning (convolutional neural networks) models using Python for classification of image patches
#Project Title:Detection of Lymph Node Metastasis in Oral Squamous Cell Carcinoma Using Deep Learning on Histopathology Images
#Author: SNB, NC

In [ ]:
# 01
# Importing necessary packages
import pandas as pd
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import RMSprop
import cv2 #For OPENCV and .tiff

#all imports succesfully installed Jan12, 2024, 02.08pm

#tensorflow-2.15.0 incompatiable with latest python 3.12, this was solved under the supervision of Nisha Ma'am by downgrading 3.12 to 3.10 in our deignated 
#conda environment using help from Google Bard. ChatGPT lacks knowledge beyond Jan2024


2024-02-05 15:55:13.065861: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#21

# Check if any GPUs are available
print(tf.config.list_physical_devices('GPU'))

# Check if TensorFlow can access the GPU
print(tf.test.is_gpu_available())




[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


2024-02-05 15:55:16.325457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:16.341923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:16.342639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:16.344739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [3]:
# 03

#Configuring TensorFlow's GPU Memory Allocation
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


In [ ]:
# 04

#ProjectDirectory and Data Directory
project_path = '/home/Documents/TrainingCNN_LN_Met_OSCC/Traindata'  # ADD YOUR PROJECT PATH HERE
data_path = f'/home/Documents/TrainingCNN_LN_Met_OSCC/Traindata/Augmented_Data'  # ADD YOUR DATA PATH HERE


In [ ]:
# 05

# Declaring Model Type
model_type = 'Xception'  # Change this to the model you want to use for eg: InceptionResNetV2,MobileNetV2, InceptionV3,ResNet50
#each time you run a specific model you have to change this parameter here and replace the existing one with the model you want to prepare

In [ ]:
#06
# ImageDataGenerator
# color images
datagen_train = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

In [ ]:
# 07

# Training Data
train_generator = datagen_train.flow_from_directory(
    data_path,
    target_size=(256, 256),
    batch_size=32 if model_type not in ['NASNetLarge', 'NASNetMobile'] else 8,
    class_mode='binary', #changed from multiclass categrical to binary 
    subset='training',
    classes=['Normal', 'Meta']  # specify the correct order of classes
  )

Found 342923 images belonging to 2 classes.


In [ ]:
# 08

# Validation Data
valid_generator = datagen_train.flow_from_directory(
    data_path,
    target_size=(256, 256),
    batch_size=32 if model_type not in ['NASNetLarge', 'NASNetMobile'] else 8,
    class_mode='binary', #changed from multiclass categrical to binary 
    subset='validation',
    shuffle=False,
    classes=['Normal', 'Meta']  # specify the correct order of classes
   )

Found 85729 images belonging to 2 classes.


In [ ]:
#21
# Check if any GPUs are available
print(tf.config.list_physical_devices('GPU'))

# Check if TensorFlow can access the GPU
print(tf.test.is_gpu_available())

In [ ]:
#22
#Check if any GPUs are available
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#23

import tensorflow as tf
print(tf.__version__)

In [ ]:
# 12
# For the selected CNN  
if model_type == 'Xception':
    inception = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(256, 256, 3)
    )
    for layer in inception.layers:
        layer.trainable = True
    x = layers.Flatten()(inception.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inception.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])

2024-02-05 15:55:45.630441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:45.631214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:45.632016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:45.632733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-05 15:55:45.633393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [ ]:
# 19 
#Model Summary

if not os.path.exists(f'{project_path}/models/{model_type}'):
    os.makedirs(f'{project_path}/models/{model_type}')
# Model Summary
model.summary()         

In [ ]:
# 20 

# Training the model
print("------------------------------------------")
print(f'Training the model {model_type}')
print("------------------------------------------")
history = model.fit(train_generator, validation_data=valid_generator, epochs=50)
print("------------------------------------------")
print(f'Training Complete')
print("------------------------------------------")

# Saving the model experimental 24.01.2024
model.save(f'{project_path}/models/{model_type}/{model_type}.h5')
print("------------------------------------------")
print(f'Model saved')
print("------------------------------------------")

In [12]:
#seprate script to cross check training and validation accuracy
print(f'Total training accuracy: {history.history["acc"][-1]*100:.2f}%')
print(f'Total validation accuracy: {history.history["val_acc"][-1]*100:.2f}%')

Total training accuracy: 100.00%
Total validation accuracy: 100.00%


In [13]:
# 26

# Saving Training History
hist_df = pd.DataFrame(history.history)
# save to json:
hist_json_file = f'{project_path}/models/{model_type}/history.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)
# or save to csv:
hist_csv_file = f'{project_path}/models/{model_type}/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
# Predict classes To print classification report
valid_generator.reset()
Y_pred = model.predict(valid_generator, steps=valid_generator.samples // valid_generator.batch_size+1)
y_pred = np.where(Y_pred > 0.5, 1, 0)

# Get true classes
y_true = valid_generator.classes

# Print classification report
print(classification_report(y_true, y_pred, target_names=['Norm', 'Meta']))

2680/2680 [==============================] - 170s 63ms/step
              precision    recall  f1-score   support

        Norm       1.00      1.00      1.00     39737
        Meta       1.00      1.00      1.00     45992

    accuracy                           1.00     85729
   macro avg       1.00      1.00      1.00     85729
weighted avg       1.00      1.00      1.00     85729



In [ ]:
# 25

# plotting the accuracy
print("------------------------------------------")
print(f'Plotting and supplementary data')
print("------------------------------------------")
plt.figure(figsize=(10, 10))
plt.plot(history.history['acc'], label='Training Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy-InceptionResNetV2') #change model name here
plt.legend(['train', 'test'], loc='upper left')
plt.tight_layout()
plt.savefig(f'{project_path}/models/{model_type}/Accuracy.jpg')

In [ ]:
#new accuracy and loss graph 01.02.2024
import pandas as pd
import matplotlib.pyplot as plt

# Load history
# Replace 'path_to_your_file' with the path to your history.csv file
history = pd.read_csv('__.csv') #put your histiry.csv file path here

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4), dpi=300)

plt.subplot(1, 2, 1)
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy-Xception')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='center left', bbox_to_anchor=(1, 0.5))

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss-InceptionResNetV2')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [18]:
# 27
# Loading Model for Testing
loaded_model = load_model(f'{project_path}/models/{model_type}/{model_type}.h5')
outcomes = loaded_model.predict(valid_generator)
y_pred = np.argmax(outcomes, axis=1)


2680/2680 [==============================] - 171s 63ms/step


In [ ]:
#Confusion Maaaatrix with slecteted font and DPI class   labels 01.02.2024 17.51pm
# Get class levels
class_levels = valid_generator.class_indices
class_labels = list(class_levels.keys())

# Compute confusion matrix
confusion = confusion_matrix(valid_generator.classes, y_pred)

# Define labels for the confusion matrix
labels = np.array([[f"True Negative = {confusion[0][0]}", f"False Positive = {confusion[0][1]}"],
                   [f"False Negative = {confusion[1][0]}", f"True Positive = {confusion[1][1]}"]])

# Plot confusion matrix
plt.figure(figsize=(6, 4), dpi=300)
sns.heatmap(confusion, annot=labels, fmt='', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix-InceptionResNetNetV2')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()

# Save the confusion matrix
plt.savefig(f'{project_path}/models/{model_type}/Confusion_matrix.jpg', dpi=300)

In [22]:
#Correction done for Class Mismatch 17.33 01.02.2024
from sklearn.metrics import confusion_matrix

# Predict classes to print classification report
valid_generator.reset()
Y_pred = model.predict(valid_generator, steps=valid_generator.samples // valid_generator.batch_size+1)
y_pred = np.where(Y_pred > 0.5, 1, 0)

# Get true classes
y_true = valid_generator.classes

# Compute the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Print the confusion matrix
print(f'True Negatives: {tn}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')
print(f'True Positives: {tp}')

# Compute the classification report
target_names = ['Normal', 'Meta']  # Updated target names
report = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)

# Convert the report to a DataFrame
df = pd.DataFrame(report).transpose()

# Print the classification report
print(df)

# Save the classification report to a CSV file
df.to_csv(f'{project_path}/models/{model_type}/Classification_report.csv')

print("------------------------------------------")
print(f'Supplementary Data Saved')
print("------------------------------------------")

2680/2680 [==============================] - 168s 63ms/step
True Negatives: 39737
False Positives: 0
False Negatives: 0
True Positives: 45992
              precision  recall  f1-score  support
Normal              1.0     1.0       1.0  39737.0
Meta                1.0     1.0       1.0  45992.0
accuracy            1.0     1.0       1.0      1.0
macro avg           1.0     1.0       1.0  85729.0
weighted avg        1.0     1.0       1.0  85729.0
------------------------------------------
Supplementary Data Saved
------------------------------------------


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Predict probabilities for the validation dataset
valid_generator.reset()
Y_pred = model.predict(valid_generator, steps=valid_generator.samples // valid_generator.batch_size+1)

# Get true classes
y_true = valid_generator.classes

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(y_true, Y_pred)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(6, 4), dpi=300)
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic-InceptionResNetV2')
plt.legend(loc="lower right")
plt.show()

In [ ]:
####################################################################################################